In [ ]:
# --- A) IMPORT LIBRARIES ---
import json
import geopandas as gpd
# --- B) FUNCTIONS ---
from functions import *
import importlib
import functions
importlib.reload(functions)
dir(functions)

------------------------------------------------

In [50]:
# --- B) IMPORT INPUT DATA ---

# b.1 DEfine paths to input data
vector_mask_path = "./input_data_for_layer_alterator/vector/vector_pct.geojson"
rules_path = "./input_data_for_layer_alterator/operation_rules/operation_rules_C3.json"
ucp_folder = "./input_data_for_layer_alterator/ucps"
fractions_folder = "./input_data_for_layer_alterator/lc_fractions"
output_folder = "./output"

'''
add info
'''

# b.2 - Load vector mask (GeoPackage, GeoJSON, or Shapefile)
if vector_mask_path.endswith(".gpkg"):
    vector_mask_layer = "layer_name"  # specify layer name for GPKG
    gdf_mask = gpd.read_file(vector_mask_path, layer=vector_mask_layer)
elif vector_mask_path.endswith(".geojson") or vector_mask_path.endswith(".json"):
    gdf_mask = gpd.read_file(vector_mask_path)
elif vector_mask_path.endswith(".shp"):
    gdf_mask = gpd.read_file(vector_mask_path)
else:
    raise ValueError("Unsupported vector format. Use GPKG, GeoJSON, or Shapefile.")

# b.3 - Load JSON rules
with open(rules_path) as f:
    rules = json.load(f)

------------------------------------------------

In [ ]:
# Parse rules and validate them against the vector mask attributes
try:
    rule_id, parsed_info = parse_rules_from_mask(gdf_mask, rules)
except ValueError as e:
    print("Rule validation failed:", e)
    raise

# Determine what action to take based on rule_id
if rule_id == "C1":
    print(f"Rule {rule_id} detected: applying masking...")
    apply_mask_rule_all(gdf_mask, rules, ucp_folder, fractions_folder, output_folder)
    print("Masking complete. Output saved to:", output_folder)

elif rule_id in {"C2", "C3"}:
    print(f"Rule {rule_id} detected: applying percentage adjustments...")
    apply_pct_all(gdf_mask, rules, ucp_folder, fractions_folder, output_folder)
    print("Percentage adjustments complete. Output saved to:", output_folder)

else:
    print(f"No processing required for rule {rule_id}. Skipping masking and percentage adjustment.")

Rule C3: Layers set to PCT and NONE. Proceeding with PCT logic and treating NONE as 0.
Rule C3 detected: applying percentage adjustments...
F_AC none 0.0 1.0
F_S pct 0.0 1.0
F_M none 0.0 1.0
F_BS pct -30 0.7
F_G pct -30 0.7
F_TV pct 30 1.3
F_W none 0.0 1.0
Pixel sum stats: 0.99999976 1.0000002 1.0
Finished applying percentage changes and polygon-wise normalization to fraction layers.
No explicit NoData value defined in the input raster.
TCH pct 50 1.5
IMD none 0.0 1.0
No explicit NoData value defined in the input raster.
BH none 0.0 1.0
BSF none 0.0 1.0
No explicit NoData value defined in the input raster.
SVF none 0.0 1.0
Percentage adjustments complete. Output saved to: ./output


---------------------------------------------------------

In [ ]:
# --- E) Test new IMD and BSF consistency
# check_imd_bsf_consistency(output_folder)

Notes for PCT:

- Think about how to deal with 0 data -> **see "zero_handling" option**
- Test if normalisation [0-1] for each UCP layer is a good proxy to obtain a proportional increase of the UCP that is wanted to be modified by having a final UCP raster still in the range [0-1]? Maybe not, because this changes the whole range of values in the raster -> **Consider only "clip" option for the UCPs**
- Check consistency in IMD and BSF -> **Creates a separate functions "check_imd_bsf_consistency" for checking**
- Think about how to deal with no-data -> **Added additional masking in bot ucp and fractions processing**
- Test if a second normalisation [0-1] per pixel fractions array (other than a normalisation on each single raster) on the modified fractions raster is sufficient to preserve proportion by obeying the rules that sum per pixel = 1 and fractions value >=0. Maybe it's better first to normalise per pixel and then apply the second normalisation to each final raster -> **Applied a single normalization within each pixel in the mask as proportional scaling Fx/sum(all_F)**